Imports


In [2]:
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, MaxPooling2D,Conv2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Preparing Data
Here i did not do train test split as the data was insufficent



In [3]:
data_gen = ImageDataGenerator(
    samplewise_std_normalization=True,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    rescale=1./255,  
)

/usr/local/lib/python3.9/dist-packages/keras/preprocessing/image.py:1469: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  warnings.warn(


In [10]:
training_data = data_gen.flow_from_directory("/content/drive/MyDrive/Agricultural-crops",
                                             target_size=(196, 196),
                                              batch_size=32,
                                              class_mode = "categorical",
                                              )

Found 829 images belonging to 30 classes.


Getting Classes with Labels


In [11]:
training_data.class_indices

{'Cherry': 0,
 'Coffee-plant': 1,
 'Cucumber': 2,
 'Fox_nut(Makhana)': 3,
 'Lemon': 4,
 'Olive-tree': 5,
 'Pearl_millet(bajra)': 6,
 'Tobacco-plant': 7,
 'almond': 8,
 'banana': 9,
 'cardamom': 10,
 'chilli': 11,
 'clove': 12,
 'coconut': 13,
 'cotton': 14,
 'gram': 15,
 'jowar': 16,
 'jute': 17,
 'maize': 18,
 'mustard-oil': 19,
 'papaya': 20,
 'pineapple': 21,
 'rice': 22,
 'soyabean': 23,
 'sugarcane': 24,
 'sunflower': 25,
 'tea': 26,
 'tomato': 27,
 'vigna-radiati(Mung)': 28,
 'wheat': 29}

# Transfer Learning
Here i took pretrained vgg16 with imagenet weights and removed the last layer

In [4]:
vgg = VGG16(include_top = False, weights = 'imagenet',input_shape=[196,196,3])

58889256/58889256 [==============================] - 0s 0us/step


In [5]:
for layer in vgg.layers:
  layer.trainable = False

In [6]:
x = Flatten()(vgg.output)
prediction = Dense(128, activation = 'relu')(x)
prediction = Dropout(0.3)(prediction)
prediction = Dense(30,activation='softmax')(prediction)

In [7]:
vgg_model = Model(inputs = vgg.input , outputs = prediction)

In [8]:
vgg_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 196, 196, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 196, 196, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 196, 196, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 98, 98, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 98, 98, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 98, 98, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 49, 49, 128)       0     

In [9]:
vgg_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model Training

In [13]:
vgg_model.fit(
    training_data,
    steps_per_epoch=len(training_data),
    epochs=200,
    shuffle = True
    )

Epoch 1/200
26/26 [==============================] - 195s 8s/step - loss: 3.3492 - accuracy: 0.0856
Epoch 2/200
26/26 [==============================] - 16s 598ms/step - loss: 3.1256 - accuracy: 0.1255
Epoch 3/200
26/26 [==============================] - 16s 612ms/step - loss: 2.9742 - accuracy: 0.1641
Epoch 4/200
26/26 [==============================] - 16s 596ms/step - loss: 2.7789 - accuracy: 0.2147
Epoch 5/200
26/26 [==============================] - 16s 606ms/step - loss: 2.5610 - accuracy: 0.2690
Epoch 6/200
26/26 [==============================] - 16s 598ms/step - loss: 2.5538 - accuracy: 0.2654
Epoch 7/200
26/26 [==============================] - 16s 593ms/step - loss: 2.3665 - accuracy: 0.3100
Epoch 8/200
26/26 [==============================] - 16s 604ms/step - loss: 2.2749 - accuracy: 0.3281
Epoch 9/200
26/26 [==============================] - 16s 593ms/step - loss: 2.1696 - accuracy: 0.3607
Epoch 10/200
26/26 [==============================] - 16s 600ms/step - loss: 2.0753 

In [14]:
vgg_model.save('vgg.h5')

# Testing

In [15]:
test_data_gen = ImageDataGenerator(
    rescale=1./255,  
    validation_split=0.1,
)

In [23]:
test_data = test_data_gen.flow_from_directory("/content/drive/MyDrive/Agricultural-crops",
                                             target_size=(196, 196),
                                              batch_size=32,
                                              class_mode = "categorical",
                                              subset = 'validation'
                                              )

Found 72 images belonging to 30 classes.


In [24]:
predictions = vgg_model.predict_generator(test_data)


<ipython-input-24-2b69a194f323>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = vgg_model.predict_generator(test_data)


In [25]:
cm = confusion_matrix(test_data.classes, np.argmax(predictions, axis=1))
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0

In [26]:
report = classification_report(test_data.classes, np.argmax(predictions, axis=1), target_names=test_data.class_indices.keys())
print("Classification Report:")
print(report)

Classification Report:
                     precision    recall  f1-score   support

             Cherry       0.00      0.00      0.00         3
       Coffee-plant       0.00      0.00      0.00         2
           Cucumber       0.00      0.00      0.00         3
   Fox_nut(Makhana)       0.00      0.00      0.00         2
              Lemon       0.00      0.00      0.00         2
         Olive-tree       0.00      0.00      0.00         3
Pearl_millet(bajra)       0.00      0.00      0.00         3
      Tobacco-plant       0.00      0.00      0.00         3
             almond       0.00      0.00      0.00         2
             banana       0.17      0.33      0.22         3
           cardamom       0.00      0.00      0.00         2
             chilli       0.00      0.00      0.00         2
              clove       0.00      0.00      0.00         3
            coconut       0.00      0.00      0.00         2
             cotton       0.00      0.00      0.00         3


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
